In [ ]:
setwd('/home/notebook/data-analysis')
# Import database functions
if (exists('INC_DATABASE_R')) {
    rm(INC_DATABASE_R)
}
source('include/database.r')
library(plyr)
library(foreign)

In [ ]:
# Create database connection
conn <- connect()
# Retrieve patterns, such as what a story point is
patterns <- load_definitions('sprint_definitions.yml')

In [ ]:
query <- 'SELECT sprint.project_id,
          EXTRACT(day FROM MAX(${s(sprint_close)}) - MIN(sprint.start_date)) AS lifespan
          FROM gros.sprint
          GROUP BY sprint.project_id ORDER BY sprint.project_id'
time <- system.time(data <- dbGetQuery(conn, load_query(list(query=query), patterns)$query))
cat(paste("Query took", time['elapsed'], "seconds"), "\n")
query2 <- 'SELECT project.project_id,
           COUNT(issue.issue_id) AS number
           FROM gros.project JOIN gros.issue ON project.project_id = issue.project_id
           WHERE issue.type = 7 AND issue.changelog_id = 0
           GROUP BY project.project_id ORDER BY project.project_id'
time2 <- system.time(data2 <- dbGetQuery(conn, load_query(list(query=query2), patterns)$query))
cat(paste("Query took", time2['elapsed'], "seconds"), "\n")
data <- join(data, data2, by="project_id")

In [ ]:
data <- data[!is.na(data$number),]
data

In [ ]:
mean(data$number / data$lifespan)

In [ ]:
sd(data$number / data$lifespan)

So each day 0.39 +- 0.30 stories are created.

In [ ]:
sprint_features <- read.arff('output/sprint_features.arff')

In [ ]:
mean(sprint_features$num_removed_stories)

In [ ]:
sd(sprint_features$num_removed_stories)

So each sprint 0.80 +- 1.94 stories are moved to another sprint.

In [ ]:
mean(sprint_features$backlog_dismiss_velocity)

In [ ]:
sd(sprint_features$backlog_dismiss_velocity)